# Value Lost

A notebook to determine the value lost when applying a maintenance strategy defined in an asset model to the population.

Requested: greg.bell@essentialenergy.com.au
Author: gavin.treseder@essentialenergy.com.au

## Project Setup

In [2]:
import copy
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from IPython.display import clear_output
import numpy as np 
import pandas as pd
import plotly.express as px
import scipy.stats as ss

from pof.system import System
from pof.interface import figures
from pof.interface.figures import make_sensitivity_fig
from pof.loader.asset_model_loader import AssetModelLoader
from pof.paths import Paths
from pof.units import scale_units_val

%matplotlib inline

# Load Data
## Load the Asset Model
The asset model needs to be loaded from the excel file and converted into a component object that execute simulations

In [3]:
asset_model_filename = "Asset Model.xlsx"

paths = Paths()
model_path = paths.demo_path + os.sep + asset_model_filename

try:
    aml = AssetModelLoader()
    pof_data = aml.load(model_path)
    system = System.from_dict(pof_data['overhead_network'])
    comp = system.comp['pole']
    print("Asset Model loaded")
    pass
except:
    comp = Component.demo()
    print("Demo Model loaded")

Asset Model loaded


In [4]:
def rolling_mean(df, window=5, x_var:str, y_var:str):
    """ Returns a df with a rolling mean"""
    #TODO set this up so it work for 
    df_results = (
        df.copy(deep=True)
        .rename(columns={'task':'source'})
        .dropna()
    )

    # Generate a rolling average
    df_rolling = (
        df_results
        .sort_values(by=['source', x_var, y_var])
        .groupby(by=['source', y_var]).rolling(window).mean()
        .drop([y_var], axis=1)
        .reset_index()
        .dropna()
    )

    df_rolling['active'] = df_rolling['active'].astype(bool)

    return df_rolling


In [5]:
def calc_value_lost(df, x_var:str, y_var:str):
    """ Takes a df of risk costs and returns a df with teh value lost
    # TODO move this to risk cost
    # TODO check y_var and x_var are the right way around
    """

    # Calculate the total cost for each scenario
    df = df_rolling.groupby(by=[x_var, y_var])['quantity', 'cost', 'quantity_annual', 'cost_annual'].sum()

    # Calculate the value_lost at each interval
    df['value_lost'] = df['cost'] - df.groupby(by=[y_var])['cost'].min()

    return df

In [6]:
def make_contour_plot(df, x_axis, y_axis, z_axis):
    """ Create a contour plot """

    df_plot = df.reset_index()

    # Arrange the data for plotly
    z = df_plot.groupby(by = [x_axis])[z_axis].apply(lambda x: x.values.tolist()).tolist()
    x = df_plot[x_axis].unique()
    y = df_plot[y_axis].unique()

    # Generate the figure
    fig = go.Figure(
        data = go.Contour(x=x, y=y, z=z, contours_coloring='lines'),
    )
    return fig

## Long run simulation 
Run a long simulation to repeat the process above for the key outputs without the interim steps

In [12]:
# Set the parameters in the model_units
model_units = 'months'
t_end = 1200
n_iterations = 100
sens_vars = {
    'pole-consequence-cost': np.arange(0, 2e6, 5e5),
    'pole-task_group_name-groundline-t_interval': np.arange(0, 120, 30)
}

x_axis = 'pole-task_group_name-groundline-t_interval'
y_axis = 'pole-consequence-cost'
z_axis = 'value_lost'

window=5

In [11]:
# Run the simulation
comp.units = model_units

df_sens_chain = comp.sensitivty_chain(sens_vars, t_end=t_end, n_iterations=n_iterations)

df_rolling = rolling_mean(df_sens_chain, x_var=x_axis, y_var=y_axis, window=window)

df_total = calc_value_lost(df=df_rolling, x_var=x_axis, y_var=y_axis)

KeyboardInterrupt: 

In [ ]:
make_contour_plot(df_total, x_axis=x_axis, y_axis=y_axis, z_axis=z_axis)

In [ ]:
df_total.to_csv("interim_model_results.csv")

# Match Asset Models to the population
## Load the population data

In [7]:
filename = r"C:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\inputs\csvs\ACS - Poles - Consequence Model Output.csv"
print("File Loading...")
df_cons = pd.read_csv(filename)
clear_output()
print("File Loaded")
df_cons.head()

File Loaded


,Unnamed: 0,ASSET_ID,Depot (ZSG Area),Pole_LU_NSWMajo,Pole_LU_NSWDeta,Pole_LU_ALUMMaj,Pole_LU_ALUMDet,Road Crossing Conductor Type,HV_LV Status,BushfirePriority,...,C_Environment_Rank,Site_Bushfire_Moderate_Probability,Site_Bushfire_Severe_Probability,C_Bushfire_Dollars,Site_Environment_Insignificant_Probability,Site_Environment_Minor_Probability,C_Environment_Dollars,C_Financial_Dollars,Total Consequence $,Radial(Y/N)
0,0,388,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2802.19134,21198.144867,NaN
1,1,389,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2802.57978,21198.533307,NaN
2,2,390,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2803.09692,21199.050447,NaN
3,3,391,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2803.76382,21199.717347,NaN
4,4,392,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2804.33010,21200.283627,NaN


In [14]:
# TODO turn ino a function so it works for mutliple y vars

pop_var = 'Total Consequence $'
sens_var = 'pole-consequence-cost'
df_sens_var = pd.DataFrame(sens_vars[sens_var], columns= [sens_var])

# Fill with average val
df_cons[pop_var] = df_cons[pop_var].fillna(df_cons[pop_var].mean())

# Match to the closest value from the model
df = pd.merge_asof(
    df_cons.sort_values(pop_var),
    df_sens_var,
    left_on = pop_var,
    right_on = sens_var,
    direction = 'forward'
)

# Group by subpopulation #TODO make it work for multiple y vars
df_pop = df.groupby(by=['Depot (ZSG Area)', y_var])['ASSET_ID'].count().reset_index()
df_pop.head()

In [ ]:
df_sens_var = pd.DataFrame(sens_vars[y_var], columns= [y_var])

# Fill with average val
df_cons[pop_var] = df_cons[pop_var].fillna(df_cons[pop_var].mean())

# Match to the closest value from the model
df = pd.merge_asof(
    df_cons.sort_values(pop_var),
    df_sens_var,
    left_on = pop_var,
    right_on = y_var,
    direction = 'forward'
)

df_plot = pd.merge(df, df_total.reset_index(), on=y_var).groupby(by=x_var).sum().reset_index()
#y= ['list(set(df_plot) - set([sens_var, uid_col]))']
y = ['quantity_annual', 'cost_annual', 'value_lost']

px.line(df_plot, x=x_var, y=y)